In [2]:
import pandas as pd

In [3]:
def read_dict(filename):
    d = dict()
    with open(filename, "r") as f:
        for line in f:
            line = line.strip().split("\t")
            d[line[0]] = int(line[1])
    return d
def write_dict(filename, d):
    with open(filename, "w") as f:
        for k, v in d.items():
            f.write(k)
            f.write("\t")
            f.write(str(v))
            f.write("\n")

In [421]:
df1 = pd.read_csv("~/Downloads/data/WK3l-15k/en_de/p_en_v6.csv", sep="@@@", header=None)
df1.columns = ["head", "relation", "tail"]
df2 = pd.read_csv("~/Downloads/data/WK3l-15k/en_fr/p_en_v5.csv", sep="@@@", header=None)
df2.columns = ["head", "relation", "tail"]

/Users/sean/miniconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/Users/sean/miniconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


In [422]:
df = pd.merge(df1, df2, how="outer")

In [423]:
df1.shape, df2.shape, df.shape

((210820, 3), (203502, 3), (370046, 3))

In [424]:
df

,head,relation,tail
0,alabama,name,alabama
1,alabama,officiallang,english
2,alabama,widthus,190
3,alabama,lowestelev,0
4,alabama,lowestelevus,0
...,...,...,...
370041,Ștefan kovács,manageryears,1980
370042,Ștefan kovács,name,Ștefan kovács
370043,Ștefan kovács,place of birth,romania
370044,Ștefan kovács,place of death,romania


In [425]:
df["head"] = df["head"].apply(str)
df["relation"] = df["relation"].apply(str)
df["tail"] = df["tail"].apply(str)

entities = sorted(set(df["head"]) | set(df["tail"]))
relations = sorted(set(df["relation"]))

entity_dict = {x: i for i, x in enumerate(entities)}
relation_dict = {x: i for i, x in enumerate(relations)}

df["head"] = df["head"].apply(lambda x: entity_dict.get(x, -1))
df["tail"] = df["tail"].apply(lambda x: entity_dict.get(x, -1))
df["relation"] = df["relation"].apply(lambda x: relation_dict.get(x, -1))

write_dict("../data/WK3l-15k_EN/entity_dict.txt", entity_dict)
write_dict("../data/WK3l-15k_EN/relation_dict.txt", relation_dict)

In [426]:
df = df.drop_duplicates()
df.to_csv("../data/WK3l-15k_EN/triple_id.txt", sep="\t", index=False, header=None)

In [427]:
import random

random.seed(0)

indices = list(range(0, df.shape[0]))
random.shuffle(indices)
valid_size = test_size = df.shape[0] // 5
train_size = df.shape[0] - valid_size - test_size

train = df.iloc[indices[:train_size]]
valid = df.iloc[indices[train_size:train_size+valid_size]]
test = df.iloc[indices[train_size+valid_size:]]


In [428]:
train.to_csv("../data/WK3l-15k_EN/train_triple_id.txt", sep="\t", index=False, header=None)
valid.to_csv("../data/WK3l-15k_EN/valid_triple_id.txt", sep="\t", index=False, header=None)
test.to_csv("../data/WK3l-15k_EN/test_triple_id.txt", sep="\t", index=False, header=None)

In [407]:
teacher_dict = read_dict("../data/CN3l_EN/entity_dict.txt")
student_dict = read_dict("../data/CN3l_FR/entity_dict.txt")

In [408]:
df1 = pd.read_csv("~/Downloads/data/CN3l/en_fr/en2fr_cn.csv", sep="@@@", header=None)
df1.columns = ["teacher", "student"]
df2 = pd.read_csv("~/Downloads/data/CN3l/en_fr/fr2en_cn.csv", sep="@@@", header=None)
df2.columns = ["student", "teacher"]

/Users/sean/miniconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/Users/sean/miniconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


In [409]:
df1["teacher"] = df1["teacher"].apply(lambda x: teacher_dict.get(x, -1))
df1["student"] = df1["student"].apply(lambda x: student_dict.get(x, -1))
df2["teacher"] = df2["teacher"].apply(lambda x: teacher_dict.get(x, -1))
df2["student"] = df2["student"].apply(lambda x: student_dict.get(x, -1))

In [410]:
df2

,student,teacher
0,5896,106
1,2932,2782
2,1259,1034
3,243,231
4,713,772
...,...,...
2141,1452,1193
2142,4560,3663
2143,5736,5016
2144,4385,3798


In [411]:
df1 = pd.DataFrame([row for row in df1.drop_duplicates().itertuples() if row[1] != -1 and row[2] != -1]).drop(columns="Index")
df2 = pd.DataFrame([row for row in df2.drop_duplicates().itertuples() if row[1] != -1 and row[2] != -1]).drop(columns="Index")

In [412]:
df1

,teacher,student
0,6202,7089
1,2839,2974
2,2942,3222
3,3017,3425
4,4500,5218
...,...,...
1904,1333,3881
1905,915,1103
1906,4814,5769
1907,3424,6089


In [413]:
df2

,student,teacher
0,5896,106
1,2932,2782
2,1259,1034
3,243,231
4,713,772
...,...,...
1794,1452,1193
1795,4560,3663
1796,5736,5016
1797,4385,3798


In [418]:
len(set(df1["teacher"]) & set(df2["teacher"])), len(set(df1["student"]) & set(df2["student"]))

(1484, 1479)

In [419]:
df = pd.merge(df1, df2, how="inner")

In [420]:
df

,teacher,student
0,6202,7089
1,2839,2974
2,2942,3222
3,4412,2184
4,5971,6860
...,...,...
1466,4646,1962
1467,5921,6820
1468,1333,3881
1469,4814,5769


In [406]:
df1.to_csv("cn3l_en_fr_aligned_entity_id.txt", sep="\t", index=False, header=None)

In [4]:
e_dict1 = read_dict("../data/DBP_DB_W/entity_dict.txt")
e_dict2 = read_dict("../data/DBP_DB_Y/entity_dict.txt")
r_dict1 = read_dict("../data/DBP_DB_W/relation_dict.txt")
r_dict2 = read_dict("../data/DBP_DB_Y/relation_dict.txt")

In [5]:
e_dict = {x: i for i, x in enumerate(sorted(set(e_dict1.keys()) | set(e_dict2.keys())))}
r_dict = {x: i for i, x in enumerate(sorted(set(r_dict1.keys()) | set(r_dict2.keys())))}

In [6]:
len(e_dict), len(r_dict)

(179271, 369)

In [6]:
write_dict("../data/DBP_DB/entity_dict.txt", e_dict)
write_dict("../data/DBP_DB/relation_dict.txt", r_dict)

In [7]:
e_dict1_rev = {v: k for k, v in e_dict1.items()}
e_dict2_rev = {v: k for k, v in e_dict2.items()}
r_dict1_rev = {v: k for k, v in r_dict1.items()}
r_dict2_rev = {v: k for k, v in r_dict2.items()}

In [8]:
df1 = pd.read_csv("../data/DBP_DB_W/test_triple_id.txt", header=None, sep="\t")
df2 = pd.read_csv("../data/DBP_DB_Y/test_triple_id.txt", header=None, sep="\t")
df1.columns = ["head", "relation", "tail"]
df2.columns = ["head", "relation", "tail"]

In [9]:
df1["head"] = df1["head"].apply(lambda x: e_dict[e_dict1_rev[x]])
df1["relation"] = df1["relation"].apply(lambda x: r_dict[r_dict1_rev[x]])
df1["tail"] = df1["tail"].apply(lambda x: e_dict[e_dict1_rev[x]])
df2["head"] = df2["head"].apply(lambda x: e_dict[e_dict2_rev[x]])
df2["relation"] = df2["relation"].apply(lambda x: r_dict[r_dict2_rev[x]])
df2["tail"] = df2["tail"].apply(lambda x: e_dict[e_dict2_rev[x]])

In [63]:
df = pd.concat([df1, df2], axis=0)

In [64]:
df.to_csv("../data/DBP_DB/test_triple_id.txt", sep="\t", index=False, header=None)

In [10]:
teacher_dict = read_dict("../data/DBP_DB/entity_dict.txt")
student_dict = read_dict("../data/DBP_WD/entity_dict.txt")

In [11]:
df1 = pd.read_csv("../data/SHARED/dbp_w_wd_aligned_entity_id.txt", sep="\t", header=None)
df1.columns = ["teacher", "student"]
df2 = pd.read_csv("../data/SHARED/dbp_y_yg_aligned_entity_id.txt", sep="\t", header=None)
df2.columns = ["teacher", "student"]

In [12]:
df1["teacher"] = df1["teacher"].apply(lambda x: e_dict2.get(e_dict1_rev[x], -1))
df2["teacher"] = df2["teacher"].apply(lambda x: e_dict1.get(e_dict2_rev[x], -1))

In [14]:
df1 = pd.DataFrame([row for row in df1.drop_duplicates().itertuples() if row[1] != -1]).drop(columns="Index")
df2 = pd.DataFrame([row for row in df2.drop_duplicates().itertuples() if row[1] != -1]).drop(columns="Index")

In [24]:
df1.shape

(20729, 2)

In [25]:
df2.shape

(20729, 2)

In [17]:
df1.to_csv("../data/SHARED/dbp_y_wd_aligned_entity_id.txt", sep="\t", index=False, header=None)
df2.to_csv("../data/SHARED/dbp_w_yg_aligned_entity_id.txt", sep="\t", index=False, header=None)

In [181]:
# student
# ent1 = [line.strip().lower().split(" ", 1)[1][1:-4].split() for line in open("../data/FB15k-237/name_en_full.txt")] 
ent1 = [line.strip().lower().split("\t", 1)[0].split("_") for line in open("../data/WK3l-15k_EN_F/entity_dict.txt")]

In [184]:
# teacher
# ent2 = [line.strip().lower().split(" ", 1)[1][1:-4].split() for line in open("../data/FB15k-237/name_en_full.txt")] 
# ent2 = [line.strip().lower().split("\t", 1)[0].split("_") for line in open("../data/WK3l-15k_EN_F/entity_dict.txt")]
ent2 = [line.strip().lower().split("\t", 1)[0][28:].split("_") for line in open("../data/DBP_DB_W/entity_dict.txt")]

In [185]:
print(len(ent1), len(ent2))

15169 100000


In [186]:
def simpson_sim(w1, w2):
    w1 = set(w1)
    w2 = set(w2)
    if len(w1) == 0 and len(w2) == 0:
        return 0
    if len(w1) == 0 or len(w2) == 0:
        return 0
    return len(w1 & w2) / min(len(w1), len(w2))

In [187]:
from tqdm import tqdm
import bisect
sims = []
ent1.sort()
ent2.sort()
ent1_str = [" ".join(e1) for e1 in ent1]
ent2_str = [" ".join(e2) for e2 in ent2]
for j, e2 in tqdm(enumerate(ent2)):
    max_i = 0
    max_sim = -1
    i = bisect.bisect_left(ent1_str, e2[0])
    while i < len(ent1):
        e1 = ent1[i]
        if e1[0] != e2[0]:
           break 
        sim = simpson_sim(e1, e2)
        if sim > max_sim:
            max_sim = sim
            max_i = i
        i += 1
    sims.append((max_i, j, max_sim))

100000it [00:00, 378833.81it/s]


In [188]:
sims.sort(key=lambda x: x[2], reverse=True)

In [189]:
ent12 = []
for i, j, sim in sims:
    if sim < 0.9:
        break
    ent12.append((i, j))

In [191]:
print(len(ent12), len(ent1), len(ent2))

12555 15169 100000


In [192]:
len(set([x[0] for x in ent12])), len(set([x[1] for x in ent12]))

(1553, 12555)

In [193]:
with open("../data/SHARED/dbp_w_wk3l-15k_en_f_aligned_entity_id.txt", "w") as f:
    for x in ent12:
        f.write("%d\t%d\n" % (x[1], x[0]))

In [194]:
with open("../data/SHARED/wk3l-15k_en_f_fr_aligned_entity_id.txt") as f:
    lines = f.readlines()
with open("../data/SHARED/wk3l-15k_fr_en_f_aligned_entity_id.txt", "w") as f:
    for line in lines:
        line = line.split()
        f.write("%s\t%s\n" % (line[1], line[0]))

In [195]:
df1 = pd.read_csv("../data/SHARED/wk3l-15k_fr_en_f_aligned_entity_id.txt", header=None, sep="\t")
df2 = pd.read_csv("../data/SHARED/dbp_w_wk3l-15k_en_f_aligned_entity_id.txt", header=None, sep="\t")
df3 = pd.read_csv("../data/SHARED/fb15k_wk3l-15k_en_f_aligned_entity_id.txt", header=None, sep="\t")

In [196]:
df1.columns = ["teacher", "student"]
df2.columns = ["teacher", "student"]
df3.columns = ["teacher", "student"]

In [198]:
print(df1.shape, df2.shape, df3.shape)

(2496, 2) (12555, 2) (3320, 2)


In [199]:
df = pd.concat([df1, df2, df3], axis=0)
print(df.shape)

(18371, 2)


In [200]:
len(set(df["student"]))

4155